# Genotype imputation and phasing

This notebook showcases our software [MendelImpute.jl](https://github.com/biona001/MendelImpute), the fastest and most memory-efficient phasing/imputation software as of 2020. Our paper will be available on arxiv soon.

## Package installation

In [1]:
# machine information for reproducibility
versioninfo()

Julia Version 1.5.0
Commit 96786e22cc (2020-08-01 23:44 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


### Add necessary packages if you haven't already

In [2]:
# using Pkg
# Pkg.add(PackageSpec(url="https://github.com/OpenMendel/SnpArrays.jl.git"))
# Pkg.add(PackageSpec(url="https://github.com/OpenMendel/VCFTools.jl.git"))
# Pkg.add(PackageSpec(url="https://github.com/OpenMendel/MendelImpute.git"))
# Pkg.add(["GeneticVariation", "Random", "SparseArrays", "Plots"])

### Load necessary packages

In [3]:
using MendelImpute
using VCFTools
using GeneticVariation
using Random
using SparseArrays
using Plots

┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1278


# What is imputation and phasing?

+ Turning SNP array (GWAS) data into sequence data
+ Inputs: 
    + Unphased (entries 0, 1, 2) genotypes at ~1 million markers 
    + A reference haplotype panel (entries 0, 1) at $\ge 40$ million markers
+ Output:
    + Phased genotypes at all markers (if genotype is 1, you know which chromosome it is on)

## Example data

Using [msprime](https://msprime.readthedocs.io/en/stable/) and our [VCFTools.jl](https://github.com/OpenMendel/VCFTools.jl) package, the `simulate_data.ipynb` notebook simulated data for this demonstration. The result is stored in 3 files:

+ **`ref.excludeTarget.vcf.gz`**: haplotype reference panel with 2400 samples on 36063 SNPs.
+ **`target.typedOnly.masked.vcf.gz`**: unphased target genotypes with 100 samples, 5000 SNPs, and 1% genotypes missing at random.
+ **`target.full.vcf.gz`**: complete genotype information (for checking imputation accuracy).

# Imputation and phasing

Now we illustrate how to use `MendelImpute.jl` for phasing and imputation. 

## Step 1: Generate `.jlso` compressed reference panel

MendelImpute requires one to pre-process the reference panel for faster reading. This is achieved via the [compress_haplotypes](https://openmendel.github.io/MendelImpute/dev/man/api/#MendelImpute.compress_haplotypes) function.

In [4]:
reffile = "./data/ref.excludeTarget.vcf.gz"
tgtfile = "./data/target.typedOnly.masked.vcf.gz"
outfile = "./data/ref.excludeTarget.jlso"
@time compress_haplotypes(reffile, tgtfile, outfile)


importing reference data...100%|████████████████████████| Time: 0:00:14


 30.240712 seconds (211.01 M allocations: 14.880 GiB, 9.61% gc time)


## Step 2: Run MendelImpute!

Note: First run will be slower. Run twice for more accurate timing result. 


In [12]:
tgtfile = "./data/target.typedOnly.masked.vcf.gz"
reffile = "./data/ref.excludeTarget.jlso"
outfile = "./data/imputed.vcf.gz"
phase(tgtfile, reffile, outfile = outfile);

Number of threads = 1
Importing reference haplotype data...
Total windows = 16, averaging ~ 708 unique haplotypes per window.

Timings: 
    Data import                     = 0.78225 seconds
        import target data             = 0.585177 seconds
        import compressed haplotypes   = 0.197073 seconds
    Computing haplotype pair        = 0.320085 seconds
        BLAS3 mul! to get M and N      = 0.0306708 seconds per thread
        haplopair search               = 0.268542 seconds per thread
        initializing missing           = 0.00435455 seconds per thread
        allocating and viewing         = 0.016047 seconds per thread
        index conversion               = 0.000103686 seconds per thread
    Phasing by win-win intersection = 0.0605121 seconds
        Window-by-window intersection  = 0.00109788 seconds per thread
        Breakpoint search              = 0.0581405 seconds per thread
        Recording result               = 0.00101006 seconds per thread
    Imputation     

## Step 3: Check MendelImpute's imputation accuracy!

In [6]:
# import complete genotype info
Xtrue = convert_gt(Float64, "./data/target.full.vcf.gz")
Ximputed = convert_gt(Float64, "./data/imputed.vcf.gz")
err = sum(Xtrue .!= Ximputed) / size(Xtrue, 1) / size(Xtrue, 2)
println("error rate = $err")

error rate = 0.0007245653439813659


## Step 4: Compare result against Beagle 5.1

We also compress the reference panel into Beagle's special format, then run [beagle 5.1](http://faculty.washington.edu/browning/beagle/beagle.html), and check error rate. This requires Java version 8. 

In [7]:
###### compress reference panel to bref3 (beagle's special reference format)
# run(pipeline(`java -jar bref3.18May20.d20.jar ./data/ref.excludeTarget.vcf.gz`, stdout="./data/ref.excludeTarget.bref3")) 

###### run beagle 5.1
# run(`java -jar beagle.18May20.d20.jar gt=data/target.typedOnly.masked.vcf.gz ref=data/ref.excludeTarget.bref3 out=data/beagle.result nthreads=1`)

###### calculate beagle 5 error rate
# Xbeagle = convert_gt(Float32, "data/beagle.result.vcf.gz")
# err = sum(Xbeagle .!= Xtrue) / size(Xtrue, 1) / size(Xtrue, 2)
# println("\nbeagle error = $err")

beagle.18May20.d20.jar (version 5.1)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.18May20.d20.jar" to list command line argument
Start time: 04:33 PM PDT on 14 Sep 2020

Command line: java -Xmx3641m -jar beagle.18May20.d20.jar
  gt=data/target.typedOnly.masked.vcf.gz
  ref=data/ref.excludeTarget.bref3
  out=data/beagle.result
  nthreads=1

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,400
Study samples:             100

Window 1 (1:36-4999868)
Reference markers:      36,063
Study markers:           5,000

Burnin  iteration 1:           3 seconds
Burnin  iteration 2:           2 seconds
Burnin  iteration 3:           2 seconds
Burnin  iteration 4:           2 seconds
Burnin  iteration 5:           2 seconds
Burnin  iteration 6:           14 seconds

Phasing iteration 1:           30 seconds
Phasing iteration 2:           2 seconds
Phasing iteration 3:           2 seconds
Phasing iteration 4:           2 seconds
Phasing iteration 5:      

### Remarks:
+ MendelImpute is ~50x faster than Beagle 5.1 (on 1 thread)
+ MendelImpute's error rate is 0.000724, so we expect roughly 1 error out of every 1400 SNPs. Beagle has roughly 4x better error rate than our. (Error is measured in getting the correct minor allele count.)
+ Genotypes **are not** pre-phased

## Post-analysis: Import, manipulate, and visual data

OpenMendel's `VCFTools.jl` allow us to import VCF files directly as numeric matrix:

In [8]:
# read data as haplotypes or genotypes
H = convert_ht(Bool, "./data/ref.excludeTarget.vcf.gz")
Ximputed = convert_gt(Float64, "./data/imputed.vcf.gz")

100×36063 Array{Union{Missing, Float64},2}:
 0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  …  0.0  0.0  1.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  2.0  0.0     0.0  0.0  2.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  2.0  0.0     0.0  0.0  0.0  1.0  0.0  2.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  2.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  1.0  0.0  …  1.0  0.0  1.0  1.0  0.0  2.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  2.0  0.0     0.0  0.0  2.0  0.0  0.0  2.0  0.0
 1.0  1.0  0.0  0.0  0.0  0.0  2.0  0.0     0.0  0.0  2.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  2.0  0.0     0.0  0.0  1.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  2.0  0.0     0.0  0.0  1.0  0.0  1.0  1.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  2.0  0.0  …  0.0  0.0  0.0  1.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  0.0  2.0  0.0     0.0  0.0  0.0  0.0  0.0  2.0  0.0
 0.0  1.0  0.0  0.0 

In [9]:
H[1:10, 1:10] * rand(10)
Ximputed[1:2:10, 1:10] * rand(10)

5-element Array{Union{Missing, Float64},1}:
 1.605655253437708
 2.436207464005956
 1.9695182100969622
 1.917729324779624
 2.436207464005956

In [10]:
size(Xtrue)

(100, 36063)

Let's visualize where did our imputation went wrong:

In [11]:
disagreeing_entries = randn(size(Xtrue)) .* sparse(Xtrue .!= Ximputed)
spy(disagreeing_entries, size=(40000, 400), 
    title="Visualize imputation error (dots = imputed incorrectly)", 
    title_location=:left, markersize = 3)

# Conclusion & package feature

+ OpenMendel offers a fast and intuitive pipeline for genotype imputation and phasing
+ After imputation, your data is immediately ready for statistical analysis and visualization

# Other package features

+ Built-in support for imputing genotypes stored in VCF files (.vcf, .vcf.gz) or PLINK files.
+ Out-of-the-box multithreaded (shared memory) parallelism.
+ Admixture estimation, with code examples to make pretty plots!
+ Ultra-compressed file for phased genotypes.
+ Imputation on dosage data

All code is on GitHub: https://github.com/OpenMendel/MendelImpute.
